In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
from pathlib import Path
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Annotated , List , Dict
from dotenv import load_dotenv
load_dotenv()


True

In [9]:
layout_json = """
{
  "image_path": "./images/landing.png",
  "image_size": {"width": 1920, "height": 1080},
  "bbox_format": "normalized_xyxy",
  "elements": [
    {"label": "Navbar", "bbox": [0.0, 0.0, 1.0, 0.08], "texts": []},
    {"label": "Hero", "bbox": [0.05, 0.12, 0.95, 0.45], "texts": []}
  ],
  "unassigned_text": [
    {"text": "Welcome to Protein Co.", "bbox": [[0.1,0.1],[0.2,0.1],[0.2,0.12],[0.1,0.12]]}
  ]
}
""".strip()

filename = "landing.png"

components = {
  "landing.png": """import Link from "next/link";
import { Button } from "@/components/ui/button";
const App = () => (<div>Old landing placeholder</div>);
export default App;"""
}

palette = ["#0f172a", "#3b82f6", "#22c55e"]

build = [
  {
    "role": "system",
    "content": "YOU ARE AN EXPERIENCED WEB DEVELOPER"  # the static system prompt from apiinference.py
  },
  {
    "role": "user",
    "content": (
      "You are an automated UI generator. The user provided a sketch of their landing page.\n"
      "You cannot see the image; instead you get a structured layout JSON.\n"
      "- All bounding boxes are normalized to [0,1] coordinates in xyxy format.\n"
      "- Each detection item has a label, bbox, and any OCR text attached when it sits inside.\n"
      "- There is also a list of unassigned OCR texts that did not fall inside any box.\n\n"
      "Your task:\n"
      "- Generate a next-js WEB landing page component (NO next-js Native).\n"
      "- Use TailwindCSS classes for styling.\n"
      "- Assume a white background overall.\n"
      "- Some text may begin with 'Note:' , do not add this text literally , take note of what the user wants to say and implement it with Grace"
      "- Group elements with similar vertical positions into semantic sections (header, main, etc.).\n"
      "- Use the element labels and attached texts to place content where it belongs.\n"
      "- If a label is generic (e.g., TextButton), choose sensible copy based on nearby text.\n\n"
      "UI Code Generation Rules:\n"
      "- You MUST generate valid React + TypeScript + Next.js App Router code only.\n"
      "- Prefer modern UI component libraries over raw HTML/Bootstrap.\n"
      "- You may use shadcn/ui, Mantine, NextUI, Flowbite-React, DaisyUI, or HeadlessUI.\n"
      "- Pick one library per component group for simplicity/readability; do not mix them all at once.\n\n"
      "Responsiveness & UX:\n"
      "- Mobile nav must not overflow: on small screens use a compact 4-5 icon nav (or bottom bar) and move search into a sheet/dialog.\n"
      "- Add loading skeletons for feed cards, suggestions, and sidebar instead of blank states.\n"
      "- Make sidebar/feed sections independently scrollable when tall.\n\n"
      "Production-ready output only. Every button must have a real purpose/link; avoid self-links or '#'. Route to real pages (use known page names from the layouts) or meaningful in-page anchors; if it triggers behavior, wire a plausible action with synthetic data and Recoil state where appropriate.\n"
      "- Post/CTA buttons must actually update UI state (e.g., append a post or open a form) rather than no-ops.\n"
      "- If prior page code is provided, reuse structure/styles and augment rather than replacing.\n"
      "- Also return a non-empty 'context' string summarizing the page purpose, key CTAs/links, and any reusable patterns.\n\n"
      "Hallucination / data rules:\n"
      "- Do NOT invent external API calls or network requests.\n"
      "- Do NOT invent real external image URLs from random CDNs or photography sites.\n"
      "- For images, use stable placeholder sources (e.g., `https://picsum.photos/seed/landing/800/500`) or clearly local assets (e.g., `/images/hero.png`).\n"
      "- If some content or images are ambiguous from the layout JSON, choose generic but realistic placeholders.\n\n"
      "Code output rules:\n"
      "- Output MUST be a single, default-exported React component suitable for a Next.js App Router file (e.g., `app/<route>/page.tsx`).\n"
      "- The code must compile as TypeScript without type errors.\n"
      "- Include all required imports (React, Next, component library imports, etc.).\n"
      "- Use named functions or `const ComponentName: React.FC = () => { ... }` with a default export at the end.\n"
      "- Ensure all JSX tags are properly closed.\n"
      "- Do NOT include `TODO` placeholders, pseudo-code, or omitted handlers.\n"
      "- Do NOT wrap the output in markdown or code fences.\n\n"
      "Product/experience consistency:\n"
      "- Use the layout for the current filename to drive structure.\n"
      "- If the structured layout JSON includes multiple pages, keep styling, typography, and navigation consistent with the other pages, but only implement the page that matches the current filename.\n"
      "- Keep relevant pages connected with navbar links, buttons, and consistent content/contrast, but do not implement other pages’ full layouts here.\n"
      "- Use empty space wisely by adding relevant sections such as secondary CTAs, feature highlights, testimonials, FAQs, or footers as appropriate, rather than leaving large blank areas.\n\n"
      "Content constraints:\n"
      "- There should be no mention of sketches, layout JSON, detection boxes, OCR, or any internal representation.\n"
      "- Do NOT mention that this UI was generated or that you are an AI.\n"
      "- The final UI should read like a hand-crafted, production-ready landing page.\n\n"
      "Structured layout JSON for the current page:\n"
      "{\"image_path\":\"./images/landing.png\",\"elements\":[],\"unassigned_text\":[]}\n"
      "\n\nExisting code for this page (for reference only; keep structure/style consistent, but refactor and improve where needed):\n"
      "```tsx\n<existing code here>\n```"
      "\n\nPreferred color palette for this page (keep consistent across all pages in the project):\n"
      "['#0f172a', '#3b82f6']\n\n"
      "Current filename: landing.png\n"
      "Now generate all the final Next.js pages code which includes the current file and all the files connected to it, following all of the rules above."
      "The output expects a dict of the format 'filename':'code'"
      
    )
  }
]


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_retries=2,
)

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--a10e3ad5-d6cd-4623-8156-494082e0ff0a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 33, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 23}})

In [18]:
class ModelOutput(BaseModel):
    code: Dict[str, str]  # filename -> code
    context: Annotated[
        str,
        "Give context about the page , what is included , what its about , about the product , about the links added . Give any context that could be generalised and used over every other page in the website",
    ]

model = llm.with_structured_output(ModelOutput)
"""Invoke the model with the structured layout JSON and return the generated code."""
messages = build
result = model.invoke(messages)
# if result is None or not getattr(result, "code", None):
#     raise RuntimeError("Model returned no code. Check API key/network access or reduce prompt size.")



In [20]:
result

ModelOutput(code={}, context="This landing page component provides a modern and responsive user interface for a product or service. It features a prominent hero section with a clear call to action, a dedicated section highlighting key features, and a final call to action before the footer. Navigation is handled via a responsive header with a desktop menu and a mobile sheet menu. Key CTAs include 'Get Started Now' and 'Start Your Free Trial', both leading to plausible signup/trial pages. The design uses a consistent color palette of dark blue (`#0f172a`) and a vibrant accent blue (`#3b82f6`) for brand consistency.")